In [21]:
!pip install aicsimageio[all] --user
!pip install bioformats_jar

  Using cached Pillow-10.0.0-cp39-cp39-win_amd64.whl (2.5 MB)
  Using cached bfio-2.3.0-py2.py3-none-any.whl (43 kB)
  Using cached mrc-0.3.1-py3-none-any.whl (27 kB)
  Using cached nd2-0.7.2-py3-none-any.whl (74 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
devbio-napari 0.7.1 requires cellpose-napari, which is not installed.
devbio-napari 0.7.1 requires napari-process-points-and-surfaces, which is not installed.


In [22]:
!pip install aicspylibczi>=3.1.1 fsspec>=2022.8.0

In [1]:
from aicsimageio.writers import OmeTiffWriter
from aicsimageio import AICSImage
from skimage.io import imshow
import os

import numpy as np

In [12]:
from aicspylibczi import CziFile

In [63]:
%%time
# Path to your CZI file
# czi_file_path = '../../test/2023_05_22_Pyl-OB_010_small.czi'

czi_file_path = '../../230518_ML_pyl_OB/2023_05_18_Pyl-OB_001.czi'

czi = CziFile(czi_file_path)

# Get the shape of the data, the coordinate pairs are (start index, size)
# dimensions = czi.dims_shape()  #  [{'X': (0, 1900), 'Y': (0, 1300), 'Z': (0, 60), 'C': (0, 4), 'S': (0, 40), 'B': (0, 1)}]

print(czi.dims)

print(czi.size)  #  (1, 40, 4, 60, 1300, 1900)
size=czi.size

czi.is_mosaic()

name = "2023_05_18_Pyl-OB_001"

for z in range(size[2]):

    # czi.get_dims_shape()
    #  working but slow
    mosaic_data = czi.read_mosaic(C=0, scale_factor=1, Z=z)
    mosaic_data.shape


    output_folder = name+'/'

    if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    mosaic_data_shape = mosaic_data.shape

    subpart_size = [1600,1920]
    # Calculate the number of subparts in each dimension
    num_subparts_x = mosaic_data_shape[2] // subpart_size[0]
    num_subparts_y = mosaic_data_shape[3] // subpart_size[1]

    for x in range(num_subparts_x):
        for y in range(num_subparts_y):
            # Extract the subpart
            subpart = mosaic_data[:, :, x * subpart_size[0]:(x + 1) * subpart_size[0],
                                  y * subpart_size[1]:(y + 1) * subpart_size[1], :]

            # Create an AICSImage object
            aics_image = AICSImage(subpart)

            # Define the output file path
            output_file = os.path.join(output_folder, f"{name}_subpart_z-{z}_x-{x}_y-{y}.ome.tiff")

            # Save the subpart as an OME-TIFF
            # aics_image.save(output_file)

            # Save the subpart as a TIFF file
            OmeTiffWriter.save(np.squeeze(subpart), output_file, dim_order="YXS")

SCZMYXA
(1, 1, 49, 1267, 1200, 1600, 3)
CPU times: total: 7h 40min 11s
Wall time: 5h 3min 55s


In [64]:
from tqdm import tqdm
import time

def nested_for_loops():
    # Define the ranges for your loops
    range_outer = range(3)
    range_middle = range(4)
    range_inner = range(5)
    
    # Calculate the total number of iterations
    total_iterations = len(range_outer) * len(range_middle) * len(range_inner)
    
    # Initialize the tqdm progress bar
    progress_bar = tqdm(total=total_iterations, desc="Processing")
    
    for i in range_outer:
        for j in range_middle:
            for k in range_inner:
                # Simulate some work
                time.sleep(0.1)  # Replace with your actual work
                
                # Update the progress bar
                progress_bar.update(1)
    
    # Close the progress bar when the loops are done
    progress_bar.close()

# Call the function
nested_for_loops()

Processing: 100%|██████████████████████████████████████████████████████████████████████| 60/60 [00:06<00:00,  9.78it/s]


In [41]:
%%time
# Path to your CZI file
czi_file_path = '../../test/2023_05_22_Pyl-OB_010_small.czi'

# Open the CZI file using aicsimageio
czi_image = AICSImage(czi_file_path, reconstruct_mosaic=False)

# Define the size of the subparts
subpart_width = 1920
subpart_height = 1600

# Get the shape of the CZI file
#image_shape = czi_image.shape

# Get a list valid scene ids
print(czi_image.scenes)

# Or by scene index
czi_image.set_scene(0)
print(czi_image.current_scene_index)

# czi_image.dims  # returns a Dimensions object
dim_order = czi_image.dims.order  # returns string "TCZYX"
print(dim_order)
# czi_image.dims.X  # returns size of X dimension
#czi_imageg.shape  # returns tuple of dimension sizes in TCZYX order
print(czi_image.shape)



# Extract the dimensions from the shape
#num_channels, num_zstacks, image_height, image_width = image_shape
num_channels = czi_image.dims.C
num_zstacks = czi_image.dims.Z
image_height = czi_image.dims.Y
image_width = czi_image.dims.X

print("X: {}, Y: {}, Z: {}, C: {}".format(image_width,image_height,num_zstacks,num_channels))

('ScanRegion0',)
0
MTCZYXS
(42, 1, 1, 49, 666, 872, 3)
X: 872, Y: 666, Z: 49, C: 1
CPU times: total: 297 ms
Wall time: 285 ms


In [4]:
%%time
# Iterate through the CZI file to read and save subparts
# for channel in range(num_channels):
for zstack in range(num_zstacks):
    for x in range(0, image_width, subpart_width):
        for y in range(0, image_height, subpart_height):
            
            # print("X: {}, Y: {}, Z: {}".format(subpart_width,subpart_height,zstack))
            
            # Read the subpart
            # subpart = czi_image.get_image_dask_data(c=1, z=zstack, x=x, y=y, size_x=subpart_width, size_y=subpart_height, size_z=1)
            # subpart = czi_image.get_image_data("YX", T=0, Z=zstack)
            subpart = czi_image.get_image_dask_data("YX", T=0, Z=zstack)
            
            
            # print(subpart.shape)

            # Define the output directory and filename
            output_dir = 'output/'
            os.makedirs(output_dir, exist_ok=True)
            output_filename = f'_zstack_{zstack}_x_{x}_y_{y}.ome.tif'
            output_path = os.path.join(output_dir, output_filename)

            # Save the subpart as a TIFF file
            OmeTiffWriter.save(subpart, output_path)
            # imshow(subpart)

print("Subparts saved successfully!")

Subparts saved successfully!
CPU times: total: 14.7 s
Wall time: 12.3 s
